In [3]:
import os    
import pandas as pd
import ollama
import chromadb

In [67]:
def parse(text):
    abs = text.split("\\\\")[2].strip()
    auth = text.split("\\\\")[1].strip().split("Comment")[0].split(":")[-1]
    print(auth)
    return abs,auth


In [68]:
ids = []
abstracts = []
authors = []
years = ["1992","1993","1994"]
cnt = 0

In [69]:
for folder in os.listdir('cit-HepTh-abstracts'):
    if folder==".DS_Store":
        continue
    # if folder not in years:
    #     continue
    # if cnt > 10:
    #     break
    for file in os.listdir('cit-HepTh-abstracts/'+folder):
        with open('cit-HepTh-abstracts/'+folder+'/'+file) as f:
            ids.append(file.split(".")[0])
            abstract, author = parse(f.read())
            abstracts.append(abstract)
            authors.append(author)
            # cnt += 1
            # if cnt > 10:
            #     break


 Clifford V. Johnson

 C.Bachas

 S.L. Adler, G.V. Bhanot, J.D. Weckel

 S. Hosono, A. Klemm, S. Theisen

 M. A. Clayton, L. Demopoulos, J. W. Moffat

 E. Elizalde, S. Naftulin and S.D. Odintsov

 Rinat Kedem, Barry M. McCoy, and Ezer Melzer

 T. Kobayashi

 John Harnad

 Preeti Parashar

 Bernhard Meister

 Jouko Mickelsson

 H.B. Gao and H. R\"omer

 M. A. Semenov-Tian-Shansky

 Denjoe O'Connor and C. R. Stephens

 Atushi Ishikawa

 E. Bergshoeff, J. de Boer, M. de Roo and T. Tjin

 M. Asorey, S. Carlip, and F. Falceto

 Yao-Zhong Zhang and Mark D. Gould

 I. Ya. Aref'eva and G. E. Arutyunov

 Kiyoshi Ezawa

 I. Jack, D. R. T. Jones and J. Panvel

 Lev Vaidman

 C.M. Hull, G. Papadopoulos and P.K. Townsend

 W. Lerche

 H.J. de Vega and I.L. Egusquiza

 Robert Marnelius

 Stefano Panzeri

 Avshalom C. Elitzur and Lev Vaidman

 B. Harms and Y. Leblanc

 M. Temple-Raston

 Ted Jacobson and Robert C. Myers

 C.Kobdaj and S.Thomas

 Malcolm J. Perry and Edward Teo

 L. Brink, T.H. Hansso

In [70]:
print(authors[:100])

[' Clifford V. Johnson\n', ' C.Bachas\n', ' S.L. Adler, G.V. Bhanot, J.D. Weckel\n', ' S. Hosono, A. Klemm, S. Theisen\n', ' M. A. Clayton, L. Demopoulos, J. W. Moffat\n', ' E. Elizalde, S. Naftulin and S.D. Odintsov\n', ' Rinat Kedem, Barry M. McCoy, and Ezer Melzer\n', ' T. Kobayashi\n', ' John Harnad\n', ' Preeti Parashar\n', ' Bernhard Meister\n', ' Jouko Mickelsson\n', ' H.B. Gao and H. R\\"omer\n', ' M. A. Semenov-Tian-Shansky\n', " Denjoe O'Connor and C. R. Stephens\n", ' Atushi Ishikawa\n', ' E. Bergshoeff, J. de Boer, M. de Roo and T. Tjin\n', ' M. Asorey, S. Carlip, and F. Falceto\n', ' Yao-Zhong Zhang and Mark D. Gould\n', " I. Ya. Aref'eva and G. E. Arutyunov\n", ' Kiyoshi Ezawa\n', ' I. Jack, D. R. T. Jones and J. Panvel\n', ' Lev Vaidman\n', ' C.M. Hull, G. Papadopoulos and P.K. Townsend\n', ' W. Lerche\n', ' H.J. de Vega and I.L. Egusquiza\n', ' Robert Marnelius\n', ' Stefano Panzeri\n', ' Avshalom C. Elitzur and Lev Vaidman\n', ' B. Harms and Y. Leblanc\n', ' M. Temple-

In [72]:
client = chromadb.PersistentClient(path="full_data/")
collection = client.get_collection(name="authors")

# store each document in a vector embedding database
for i, d in enumerate(authors):
  response = ollama.embeddings(model="mxbai-embed-large", prompt=d)
  embedding = response["embedding"]
  collection.add(
    ids=[ids[i]],
    embeddings=[embedding],
    documents=[d]
  )
  


In [37]:
client = chromadb.PersistentClient(path="full_data/")
collection = client.create_collection(name="docs")

# store each document in a vector embedding database
for i, d in enumerate(abstracts):
  response = ollama.embeddings(model="mxbai-embed-large", prompt=d)
  embedding = response["embedding"]
  collection.add(
    ids=[ids[i]],
    embeddings=[embedding],
    documents=[d]
  )
  


In [42]:
len(abstracts)

29555

In [21]:
client = chromadb.PersistentClient(path="full_data/")
collection = client.get_collection(name="docs")

# an example prompt
prompt = """  Effective world-brane actions for solitons of ten-dimensional type IIA and
IIB superstring theory are derived using the formulation of solitons as
Dirichlet branes. The one-brane actions are used to recover predictions of
SL(2,Z) strong-weak coupling duality. The two-brane action, which contains a
hidden eleventh target space coordinate, is shown to be the eleven-dimensional
supermembrane action. It can be thought of as the membrane action of
`M-theory'."""

# generate an embedding for the prompt and retrieve the most relevant doc
response = ollama.embeddings(
  prompt=prompt,
  model="mxbai-embed-large"
)
results = collection.query(
  query_embeddings=[response["embedding"]],
  n_results=10,
)
data = results['documents'][0][0]
data

"Effective world-brane actions for solitons of ten-dimensional type IIA and\nIIB superstring theory are derived using the formulation of solitons as\nDirichlet branes. The one-brane actions are used to recover predictions of\nSL(2,Z) strong-weak coupling duality. The two-brane action, which contains a\nhidden eleventh target space coordinate, is shown to be the eleven-dimensional\nsupermembrane action. It can be thought of as the membrane action of\n`M-theory'."

In [24]:
results

{'ids': [['9601003',
   '9603116',
   '9609004',
   '9804076',
   '0301223',
   '9512062',
   '9703118',
   '9804051',
   '9501068',
   '9602112']],
 'embeddings': None,
 'documents': [["Effective world-brane actions for solitons of ten-dimensional type IIA and\nIIB superstring theory are derived using the formulation of solitons as\nDirichlet branes. The one-brane actions are used to recover predictions of\nSL(2,Z) strong-weak coupling duality. The two-brane action, which contains a\nhidden eleventh target space coordinate, is shown to be the eleven-dimensional\nsupermembrane action. It can be thought of as the membrane action of\n`M-theory'.",
   "String and membrane dynamics may be unified into a theory of 2+2 dimensional\nself-dual world-volumes living in a 10+2 dimensional target space. Some of the\nvacua of this M-theory are described by the N=(2,1) heterotic string, whose\ntarget space theory describes the world-volume dynamics of 2+2 dimensional\n`M-branes'. All classes of stri